<a href="https://colab.research.google.com/github/hayannn/AIFFEL_STUDY/blob/main/PromptTemplate%2C_LLM_%ED%94%84%EB%A1%AC%ED%94%84%ED%8A%B8_%EC%9E%85%EB%A0%A5%EC%9D%84_%EB%8D%94_%ED%8E%B8%ED%95%98%EA%B2%8C_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 실습 준비

In [ ]:
!pip install -q openai langchain langchain-openai

In [ ]:
!pip install -U langchain-community

In [17]:
#API KEY 저장을 위한 os 라이브러리 호출
import os

#기본 LLM 로드를 위한 라이브러리 호출
from langchain.llms import OpenAI

#채팅 LLM 로드를 위한 라이브러리 호출
from langchain.chat_models import ChatOpenAI
from langchain_community.chat_models import ChatOpenAI

In [5]:
#OPENAI API키 저장
os.environ["OPENAI_API_KEY"] = 'YOUR_API_KEY'

-------

## Davinch-003 모델 설정하기 ➡️ gpt-3.5-turbo로 교체

In [ ]:
davinci3 = OpenAI(
    model_name="text-davinci-003",
    max_tokens = 1000
)

- deprecated 상태의 모델이기 때문에 gpt-3.5-turbo로 모델 교체

In [27]:
gpt3 = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    max_tokens=1000
)

<ipython-input-27-a269f493682e>:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  gpt3 = ChatOpenAI(


<br>

## 1. 프롬프트 템플릿 맛보기

프롬프트 템플릿은 크게 2가지가 존재합니다.
1. Prompt Template
2. Chat Prompt Template

1번 Prompt Template은 일반적인 프롬프트 템플릿을 생성할때 활용합니다.

2번 Chat Prompt Template은 채팅 LLM에 프롬프트를 전달하는 데에 활용할 수 있는 특화 프롬프트 템플릿입니다.



#### Prompt Template

In [7]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

#프롬프트 템플릿을 통해 매개변수 삽입 가능한 문자열로 변환
string_prompt = PromptTemplate.from_template("tell me a joke about {subject}")

#매개변수 삽입한 결과를 string_prompt_value에 할당
string_prompt_value = string_prompt.format_prompt(subject="soccer")

#채팅LLM이 아닌 LLM과 대화할 때 필요한 프롬프트 = string prompt
string_prompt_value

StringPromptValue(text='tell me a joke about soccer')

- to_string() 함수를 통해 prompt template으로 생성한 문장 raw_text 반환 가능

In [8]:
print(string_prompt_value.to_string())

tell me a joke about soccer


#### Chat Prompt Template

In [9]:
chat_prompt = ChatPromptTemplate.from_template("tell me a joke about {subject}")
chat_prompt_value = chat_prompt.format_prompt(subject="soccer")
chat_prompt_value

ChatPromptValue(messages=[HumanMessage(content='tell me a joke about soccer', additional_kwargs={}, response_metadata={})])

In [10]:
chat_prompt_value.to_string()

'Human: tell me a joke about soccer'

<br>

## 2. 프롬프트 템플릿 활용해보기

반복적인 프롬프트를 삽입해야하는 경우, Prompt Template를 통해 간편하게 LLM을 활용할 수 있습니다.

- GPT-3와 프롬프트 템플릿을 활용하여 대화해보기

In [11]:
from langchain.prompts.prompt import PromptTemplate

template = """
너는 요리사야. 내가 가진 재료들을 갖고 만들 수 있는 요리를 추천하고, 그 요리의 레시피를 제시해줘.
내가 가진 재료는 아래와 같아.

<재료>
{재료}

"""
prompt_template = PromptTemplate(
    input_variables = ['재료'],
    template = template
)

In [12]:
print(prompt_template.format(재료 = '양파, 계란, 사과, 빵'))


너는 요리사야. 내가 가진 재료들을 갖고 만들 수 있는 요리를 추천하고, 그 요리의 레시피를 제시해줘.
내가 가진 재료는 아래와 같아.

<재료>
양파, 계란, 사과, 빵




In [28]:
response = gpt3.predict(
    prompt_template.format(재료="양파, 계란, 사과, 빵")
)

print(response)

<ipython-input-28-c6f8c8e6f34e>:1: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = gpt3.predict(


제가 추천하는 요리는 "양파 계란 볶음밥"입니다. 

<양파 계란 볶음밥 레시피>
1. 양파를 다져주세요.
2. 팬에 식용유를 두르고 양파를 볶다가 계란을 풀어 넣어 볶아주세요.
3. 밥을 넣고 골고루 섞어주세요.
4. 간장, 소금, 후추로 간을 해주세요.
5. 사과를 곁들여 먹으면 더 맛있어요.
6. 빵과 함께 서빙해주세요.

이렇게 간단하면서도 맛있는 요리를 즐기세요!


## ChatGPT와 프롬프트 템플릿을 활용하여 대화해보기

In [29]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [30]:
# ChatGPT 모델 로드
chatgpt = ChatOpenAI(temperature=0)

# 역할 부여(위에서 정의한 Template 사용)
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

# 사용자가 입력할 매개변수 template 선언
human_template = "{재료}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

#ChatPromptTemplate에 system message와 human message 템플릿 삽입
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# ChatGPT API에 ChatPromptTemplate 입력 시, human message의 매개변수인 '재료'를 할당해 전달
# -> ChatGPT는 ChatPromptTemplate의 system message와 human message를 전달받아 -> 대답 생성에 활용
answer = chatgpt(chat_prompt.format_prompt(재료="양파, 계란, 사과, 빵").to_messages())
print(answer.content)

<ipython-input-30-b7fac58ea93d>:16: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = chatgpt(chat_prompt.format_prompt(재료="양파, 계란, 사과, 빵").to_messages())


제가 추천하는 요리는 "양파 계란 토스트"입니다. 이 요리는 간단하면서도 맛있는 아침 식사나 간식으로 딱 좋아요. 준비물은 양파, 계란, 빵이 필요합니다.

**양파 계란 토스트 레시피:**

**재료:**
- 빵 4조각
- 양파 1개
- 계란 2개
- 소금
- 후추
- 식용유

**만드는 법:**
1. 양파를 얇게 썰어줍니다.
2. 팬에 식용유를 두르고 양파를 볶아 투명해질 때까지 볶아줍니다.
3. 볶은 양파를 한쪽으로 밀어 팬을 비워줍니다.
4. 계란을 푼 후 소금과 후추로 간을 해줍니다.
5. 양파가 있는 쪽에 계란을 부어줍니다.
6. 계란이 익을 때까지 젓가락으로 저어가며 볶아줍니다.
7. 빵을 토스터기에 굽거나 팬에 구워줍니다.
8. 계란이 익은 후 빵 위에 올려줍니다.
9. 다른 빵 조각으로 덮어 완성합니다.

맛있는 양파 계란 토스트가 완성되었습니다. 맛있게 드세요!


<br>

## 3. Few-shot 예제를 통한 프롬프트 템플릿

- 딥러닝 모델이 결과물을 출력할 때 예시 결과물을 제시함으로써 원하는 결과물로 유도하는 방법론
- LLM에 Few-shot 예제를 제공하면 예제와 유사한 형태의 결과물을 출력함
- 내가 원하는 결과물의 형태가 특수하거나, 구조화된 답변을 원할 경우 사용
- 결과물의 예시를 여러 개 제시 ➡️ 결과물의 품질 향상 가능

In [68]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

examples = [
  {
    "question": "아이유로 삼행시 만들어줘",
    "answer":
"""
아: 아이유는
이: 이런 강의를 들을 이
유: 유가 없다.
"""
  },

  {
    "question": "김민수로 삼행시 만들어줘",
    "answer":
"""
김: 김치는 맛있다
민: 민달팽이도 좋아하는 김치!
수: 수억을 줘도 김치는 내꺼!
"""
  }
]

In [69]:
example_prompt = PromptTemplate(input_variables=["question", "answer"], template="Question: {question}\n{answer}")

print(example_prompt.format(**examples[0]))

Question: 아이유로 삼행시 만들어줘

아: 아이유는
이: 이런 강의를 들을 이
유: 유가 없다.



In [70]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"]
)

print(prompt.format(input="호날두로 삼행시 만들어줘"))

Question: 아이유로 삼행시 만들어줘

아: 아이유는
이: 이런 강의를 들을 이
유: 유가 없다.


Question: 김민수로 삼행시 만들어줘

김: 김치는 맛있다
민: 민달팽이도 좋아하는 김치!
수: 수억을 줘도 김치는 내꺼!


Question: 호날두로 삼행시 만들어줘


In [71]:
print(gpt3.predict("호날두로 삼행시 만들어줘"))

호날두는 축구 신이요
공을 다루는 그 모습이
팬들을 미치게 만들어

호날두는 레전드 선수
기술과 스피드 모두 뛰어
경기장을 빛내주네


In [81]:
from langchain_openai import ChatOpenAI

gpt3 = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    max_tokens=1000
)

messages = [
    {"role": "user", "content": "호날두로 삼행시 만들어줘"}
]

response = gpt3(messages)
print(response)

content='호날두는 축구왕,\n공을 다루는 손재주,\n승리는 그의 몫이야.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 24, 'total_tokens': 59, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-9fb785e7-b3f9-4504-a301-a1449503daa6-0' usage_metadata={'input_tokens': 24, 'output_tokens': 35, 'total_tokens': 59, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [ ]:
# print(gpt3(
#     prompt.format(input="호날두로 삼행시 만들어줘")
# ))

In [87]:
# example_prompt = PromptTemplate(input_variables=["question", "answer"], template="Question: {question}\n{answer}")

# prompt = FewShotPromptTemplate(
#     examples=examples,
#     example_prompt=example_prompt,
#     suffix="Question: {input}",
#     input_variables=["input"]
# )

# gpt3 = ChatOpenAI(
#     model_name="gpt-3.5-turbo",
#     max_tokens=1000
# )

formatted_prompt = prompt.format(input="호날두로 삼행시 만들어줘")
response = gpt3([{"role": "user", "content": formatted_prompt}])
print(response.content)

호: 호날두는
날: 날마다 골을 넣는다
두: 두 손으로 팀을 이끈다.


## 4. Example Selector를 이용한 동적 Few-shot 러닝

- `Example Selector` 활용
- LLM이 여러 작업을 수행하면서도 **원하는 범위의 대답을 출력**하도록 하려면..
  - 사용자의 입력에 동적으로 반응해야함
  - 이와 동시에, 예제를 모두 학습시키는 것이 아니라 적절한 예시만 포함하도록 함으로써 입력 prompt의 길이를 제한하고, 이를 통해 오류가 발생하지 않도록 조절할 수 있음

- `SemanticSimilarityExampleSelector`
  - 사용자 입력과 예제 사이의 유사성 비교 ➡️ 사용자 입력과 비슷한 예제부터 가져오도록 하는 것

In [90]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# These are a lot of examples of a pretend task of creating antonyms.
examples = [
    # 감정 비교
    {"input": "행복", "output": "슬픔"},
    {"input": "흥미", "output": "지루"},
    {"input": "불안", "output": "안정"},

    # 대소 단위 비교
    {"input": "긴 기차", "output": "짧은 기차"},
    {"input": "큰 공", "output": "작은 공"},
]

In [ ]:
!pip install chromadb
!pip install tiktoken

- 주어진 예시들을 바탕으로 반대 의미를 가진 단어를 출력하는 템플릿 생성
  - `example_selector` : 입력과 유사한 예시를 찾기 위해 OpenAI 임베딩과 Chroma를 사용하여 가장 비슷한 예시를 선택
  - `FewShotPromptTemplate` : 선택된 예시를 바탕으로, "반대 의미를 가진 단어"를 출력하는 프롬프트를 생성
  - 입력 : 사용자가 제공한 단어에 대해 모델이 반대 의미를 출력하도록 유도

In [91]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    Chroma,
    k=1
)
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="주어진 입력에 대해 반대의 의미를 가진 단어를 출력해줘",
    suffix="Input: {단어}\nOutput:",
    input_variables=["단어"],
)

<ipython-input-91-ea5bcf3d34df>:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  OpenAIEmbeddings(),


In [92]:
# Input is a feeling, so should select the happy/sad example
print(similar_prompt.format(단어="무서운"))

주어진 입력에 대해 반대의 의미를 가진 단어를 출력해줘

Input: 불안
Output: 안정

Input: 무서운
Output:


In [93]:
# Input is a feeling, so should select the happy/sad example
print(similar_prompt.format(단어="큰 비행기"))

주어진 입력에 대해 반대의 의미를 가진 단어를 출력해줘

Input: 긴 기차
Output: 짧은 기차

Input: 큰 비행기
Output:


In [94]:
query = "큰 비행기"

print(gpt3(
    similar_prompt.format(단어=query)
))

content='작은 비행기' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 60, 'total_tokens': 67, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-4dbc9b94-6e52-4ecd-a471-af9eee5d0d96-0' usage_metadata={'input_tokens': 60, 'output_tokens': 7, 'total_tokens': 67, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


<br>

## 5. Output Parser를 활용한 출력값 조정

- `OutputParser`
  - LLM의 답변을 내가 원하는 형태로 고정
  - 리스트, JSON 형태 등 다양한 형식의 답변을 고정하여 출력 가능

In [95]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

In [96]:
output_parser = CommaSeparatedListOutputParser()

In [97]:
format_instructions = output_parser.get_format_instructions()
format_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [98]:
prompt = PromptTemplate(
    template="{주제} 5개를 추천해줘.\n{format_instructions}",
    input_variables=["주제"],
    partial_variables={"format_instructions": format_instructions}
)

In [99]:
model = OpenAI(temperature=0)

In [103]:
_input = prompt.format(주제="영화")
output = model(_input)

In [104]:
output

'\n\n1. Parasite\n2. The Shawshank Redemption\n3. Inception\n4. The Godfather\n5. Pulp Fiction'

In [105]:
output_parser.parse(output)

['1. Parasite',
 '2. The Shawshank Redemption',
 '3. Inception',
 '4. The Godfather',
 '5. Pulp Fiction']